In [1]:
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation
from keras.layers import Dropout, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
from keras.utils import to_categorical
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

b:\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# This function retrieves the previous 4 days given a single day for input from the .csv file.
def getEvalData(nparray,date,predict_delta):
    res = [[]]
    idx = np.argwhere(nparray==date)[0][0] - 5
    # if the given date does not have 5 previous values...
    if idx < 4 or idx + 7 > nparray.shape[0]:
        return
    else:
        for i in range(5):
            res[0].append(nparray[i+idx][1])
        predict_day = nparray[i+idx+predict_delta][1]
        predict_day_percent = (predict_day - res[0][4])/predict_day
        res.append(predict_day_percent)
        res.append(predict_day)
    return res
        

In [98]:
# Trying to convert the input data, which starts off as dataframeX, must take 5 days and then transpose it
# and then convert it into a numpy array

rawdata = pd.read_csv('data/bitcoin.csv',usecols=[8])
days = 5                                                      # Number of days we are looking at
predict_delta = 7                                             # Number of days we are predicting into the future 
number_of_items = rawdata.shape[0] - (predict_delta + days)   # number of tests
# size of our test 
test_size = number_of_items - 733

# five day blocks in a list
Xtrain = []    
# the values of the day we are trying to predict
Ytrain = []                                                   



# Splits the array of closing prices up into multiple arrays, each holding five days worth of prices
# then transposes each array and finally appends them onto one large matrix
countup = 0
counteven = 0
countdown = 0
for i in range(number_of_items - 200): 
    # finding the predict day
    predict = rawdata.head(days + i + predict_delta).tail(1).as_matrix().transpose().tolist()[0][0]
    last_day = rawdata.head(days + i).tail(1).as_matrix().transpose().tolist()[0][0]
    # Converting and adding data to arrays
    res = rawdata.head(days + i).tail(days).as_matrix().transpose().tolist()[0]
    Xtrain.append(res)
    y_val = (predict - last_day)/predict
    if y_val > .03:
        countup = countup + 1
        Ytrain.append(np.array([1, 0, 0]))
        
    elif y_val > -.03 and y_val < .03:
        counteven = counteven + 1
        Ytrain.append(np.array([0, 1, 0]))
        
    else:
        countdown = countdown + 1
        Ytrain.append(np.array([0, 0, 1]))
        
    
print(countup)
print(counteven)
print(countdown)
# converts python arrays into numpy arrays
Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)
print(Xtrain)

#print(Ytrain)

576
518
439
[[ 134.21  144.54  139.    116.99  105.21]
 [ 144.54  139.    116.99  105.21   97.75]
 [ 139.    116.99  105.21   97.75  112.5 ]
 ...
 [2608.56 2518.66 2571.34 2518.44 2372.56]
 [2518.66 2571.34 2518.44 2372.56 2337.79]
 [2571.34 2518.44 2372.56 2337.79 2398.84]]


In [97]:
# number of evalutaion data points
eval_size = number_of_items - 200

# evaluation input data
XEvaluation = []
# evaluation output data
YEvaluation = []

# appending evaluation data to appropriate arrays
for i in range(eval_size):
    # finding the day we want to predict a week out
    predict = rawdata.head(days + i + test_size + predict_delta).tail(1).as_matrix().transpose().tolist()[0][0]
    
    # finding the last day out of the 5 we are using as input to later calculate the percent difference 
    # of the change.  The percentage change is not the absolute value as we want to calculate negative percentages as well
    last_day = rawdata.head(days + i + test_size).tail(1).as_matrix().transpose().tolist()[0][0]
    
    # result after running the AvgPrevious function on the 5 days used as input
    res = rawdata.head(days + i + test_size).tail(days).as_matrix().transpose().tolist()[0]
    XEvaluation.append(res)
    
    # putting y-values into the correct bucket
    #y_val = (predict - last_day)/predict
   # if y_val > .03:
    #    countup = countup + 1
   #     YEvaluation.append(np.array([1, 0, 0]))
        
   # elif y_val > -.03 and y_val < .03:
    #    counteven = counteven + 1
   #     YEvaluation.append(np.array([0, 1, 0]))
        
    #else:
        countdown = countdown + 1
        YEvaluation.append(np.array([0, 0, 1]))
    
XEvaluation = np.array(XEvaluation)
YEvaluation = np.array(YEvaluation)

print(XEvaluation)
print(YEvaluation)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [94]:
model = Sequential()

model.add(Dense(48,input_dim=5))
model.add(Activation('sigmoid'))
model.add(Dropout(1.0))
model.add(Dense(48, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

H = model.fit(Xtrain, Ytrain, epochs=10,shuffle=True)

Epoch 1/10
1533/1533 [==============================] - 1s 603us/step - loss: 1.0974 - acc: 0.3627
Epoch 2/10
1533/1533 [==============================] - 0s 132us/step - loss: 1.0948 - acc: 0.3633
Epoch 3/10
1533/1533 [==============================] - 0s 138us/step - loss: 1.0930 - acc: 0.3816
Epoch 4/10
1533/1533 [==============================] - 0s 181us/step - loss: 1.0951 - acc: 0.3673
Epoch 5/10
1533/1533 [==============================] - 0s 146us/step - loss: 1.0917 - acc: 0.3894
Epoch 6/10
1533/1533 [==============================] - 0s 129us/step - loss: 1.0886 - acc: 0.3914
Epoch 7/10
1533/1533 [==============================] - 0s 144us/step - loss: 1.0875 - acc: 0.3999
Epoch 8/10
1533/1533 [==============================] - 0s 151us/step - loss: 1.0867 - acc: 0.3875
Epoch 9/10
1533/1533 [==============================] - 0s 136us/step - loss: 1.0897 - acc: 0.4031
Epoch 10/10
1533/1533 [==============================] - 0s 133us/step - loss: 1.0845 - acc: 0.3914


In [89]:
Test = [8070.8, 8891.21, 8516.24, 9477.84, 10016.49]
XVal = np.array([Test])
r = model.predict(XVal)

print(r)

[[0.5155729  0.22694626 0.2574809 ]]


In [51]:
score = model.evaluate(XEvaluation,YEvaluation,batch_size=100, verbose=1)
test_loss = score[0]
print("The evaluation loss is: " + str(test_loss))

ValueError: Error when checking target: expected dropout_19 to have shape (3,) but got array with shape (1,)

In [87]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_84 (Dense)             (None, 48)                288       
_________________________________________________________________
activation_30 (Activation)   (None, 48)                0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 48)                0         
_________________________________________________________________
dense_85 (Dense)             (None, 48)                2352      
_________________________________________________________________
dense_86 (Dense)             (None, 24)                1176      
_________________________________________________________________
dense_87 (Dense)             (None, 3)                 75        
Total params: 3,891
Trainable params: 3,891
Non-trainable params: 0
_________________________________________________________________


In [93]:
rawdata = pd.read_csv('data/bitcoin.csv',usecols=[3,8])
data = rawdata.as_matrix()
#print(np.argwhere(data=='2016-10-08')[0][0])

res = getEvalData(data,'2018-01-21',predict_delta)

evaluation = np.array([res[0]])
print(model.predict(evaluation))
print(res)

[[0.5155729  0.22694626 0.2574809 ]]
[[11490.5, 11188.6, 11474.9, 11607.4, 12899.2], -0.12748345818000645, 11440.7]


In [92]:
model.save('classifier_working.h5')